In [26]:
!pip install SimpleITK


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import SimpleITK as sitk

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [28]:
# Define paths (update these according to your directory structure)
data_dir = '/kaggle/input/luna-16/Dataset'
subset_dirs = [os.path.join(data_dir, f'subset{i}', f'subset{i}') for i in range(10)]
annotations_path = os.path.join(data_dir, 'annotations.csv')

# Load annotations and candidates
annotations = pd.read_csv(annotations_path)
candidates = pd.read_csv("/kaggle/input/luna-16/Dataset/candidates_V2/candidates_V2.csv")

# Combine annotations with candidates (1 = nodule, 0 = non-nodule)
# We'll use the candidates_v2 file which has more comprehensive negative samples
annotations['class'] = 1
candidates['class'] = candidates['class'].map({0: 0, 1: 1})

# Create a unified dataframe
data = pd.concat([
    annotations[['seriesuid', 'coordX', 'coordY', 'coordZ', 'class']],
    candidates[['seriesuid', 'coordX', 'coordY', 'coordZ', 'class']]
])

# Function to load CT scan
def load_ct_scan(seriesuid):
    for subset_dir in subset_dirs:
        mhd_path = os.path.join(subset_dir, seriesuid + '.mhd')
        if os.path.exists(mhd_path):
            ct_scan = sitk.ReadImage(mhd_path)
            ct_array = sitk.GetArrayFromImage(ct_scan)
            return ct_array, ct_scan
    raise FileNotFoundError(f"CT scan {seriesuid} not found in any subset")
def extract_patch(ct_array, coord_x, coord_y, coord_z, spacing, patch_size=64):
    # Convert world coordinates to voxel coordinates
    vox_x = int(np.round(coord_x / spacing[0]))
    vox_y = int(np.round(coord_y / spacing[1]))
    vox_z = int(np.round(coord_z / spacing[2]))
    
    # Get array dimensions
    depth, height, width = ct_array.shape
    
    # Calculate the center of the patch
    center_z = min(max(patch_size//2, vox_z), depth - patch_size//2)
    center_y = min(max(patch_size//2, vox_y), height - patch_size//2)
    center_x = min(max(patch_size//2, vox_x), width - patch_size//2)
    
    # Calculate patch boundaries
    z_start = center_z - patch_size//2
    z_end = center_z + patch_size//2
    y_start = center_y - patch_size//2
    y_end = center_y + patch_size//2
    x_start = center_x - patch_size//2
    x_end = center_x + patch_size//2
    
    # Extract patch
    patch = ct_array[z_start:z_end, y_start:y_end, x_start:x_end]
    
    # Ensure the patch has the correct size (in case of edge cases)
    if patch.shape != (patch_size, patch_size, patch_size):
        # If the patch is smaller than expected, pad it
        pad_z = max(0, patch_size - patch.shape[0])
        pad_y = max(0, patch_size - patch.shape[1])
        pad_x = max(0, patch_size - patch.shape[2])
        
        patch = np.pad(patch, 
                      ((0, pad_z), (0, pad_y), (0, pad_x)), 
                      mode='constant', constant_values=-1000)
        
        # If we had to pad, take the correct center portion
        if pad_z > 0 or pad_y > 0 or pad_x > 0:
            patch = patch[:patch_size, :patch_size, :patch_size]
    
    return patch


In [29]:
class Luna16Dataset(Dataset):
    def __init__(self, data, transform=None, patch_size=64, sample_negative_ratio=1.0):
        self.data = data
        self.transform = transform
        self.patch_size = patch_size
        self.sample_negative_ratio = sample_negative_ratio
        
        # Balance the dataset by sampling negative examples
        pos_data = self.data[self.data['class'] == 1]
        neg_data = self.data[self.data['class'] == 0]
        
        if sample_negative_ratio < 1.0:
            neg_data = neg_data.sample(frac=sample_negative_ratio, random_state=42)
        
        self.balanced_data = pd.concat([pos_data, neg_data])
        
        # Pre-calculate spacing for each seriesuid
        self.spacing_cache = {}
        
    def __len__(self):
        return len(self.balanced_data)
    
    def __getitem__(self, idx):
        item = self.balanced_data.iloc[idx]
        seriesuid = item['seriesuid']
        coord_x = item['coordX']
        coord_y = item['coordY']
        coord_z = item['coordZ']
        label = item['class']
        
        # Get spacing from cache or load it
        if seriesuid not in self.spacing_cache:
            _, ct_scan = load_ct_scan(seriesuid)
            spacing = ct_scan.GetSpacing()
            self.spacing_cache[seriesuid] = spacing
        else:
            spacing = self.spacing_cache[seriesuid]
        
        # Load CT array if not in cache
        ct_array, _ = load_ct_scan(seriesuid)
        
        # Extract patch
        patch = extract_patch(ct_array, coord_x, coord_y, coord_z, spacing, self.patch_size)
        
        # Normalize HU values
        patch = self.normalize_hu(patch)
        
        # Add channel dimension
        patch = np.expand_dims(patch, axis=0)
        
        if self.transform:
            patch = self.transform(patch)
            
        return torch.FloatTensor(patch), torch.FloatTensor([label])
    
    def normalize_hu(self, patch):
        # Clip HU values to [-1000, 400] (lung window)
        patch = np.clip(patch, -1000, 400)
        # Scale to [0, 1]
        patch = (patch + 1000) / 1400
        return patch

In [30]:
class BasicBlock3D(nn.Module):
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(out_channels)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        residual = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            residual = self.downsample(x)
            
        out += residual
        out = self.relu(out)
        
        return out

class ChannelAttention3D(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(ChannelAttention3D, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.max_pool = nn.AdaptiveMaxPool3d(1)
        
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction_ratio),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction_ratio, in_channels),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        b, c, _, _, _ = x.size()
        
        avg_out = self.fc(self.avg_pool(x).view(b, c))
        max_out = self.fc(self.max_pool(x).view(b, c))
        
        out = avg_out + max_out
        out = out.view(b, c, 1, 1, 1)
        
        return out

class SpatialAttention3D(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention3D, self).__init__()
        assert kernel_size in (3, 7), "kernel size must be 3 or 7"
        padding = 3 if kernel_size == 7 else 1
        
        self.conv = nn.Conv3d(2, 1, kernel_size=kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        concat = torch.cat([avg_out, max_out], dim=1)
        
        out = self.conv(concat)
        out = self.sigmoid(out)
        
        return out

class CBAMBlock3D(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16, kernel_size=7):
        super(CBAMBlock3D, self).__init__()
        self.channel_attention = ChannelAttention3D(in_channels, reduction_ratio)
        self.spatial_attention = SpatialAttention3D(kernel_size)
        
    def forward(self, x):
        out = x * self.channel_attention(x)
        out = out * self.spatial_attention(out)
        return out

class ResNet3D_CBAM(nn.Module):
    def __init__(self, block, layers, num_classes=1, init_channels=64):
        super(ResNet3D_CBAM, self).__init__()
        self.in_channels = init_channels
        
        self.conv1 = nn.Conv3d(1, init_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm3d(init_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, init_channels, layers[0])
        self.layer2 = self._make_layer(block, init_channels*2, layers[1], stride=2)
        self.layer3 = self._make_layer(block, init_channels*4, layers[2], stride=2)
        self.layer4 = self._make_layer(block, init_channels*8, layers[3], stride=2)
        
        # Attention modules
        self.cbam1 = CBAMBlock3D(init_channels)
        self.cbam2 = CBAMBlock3D(init_channels*2)
        self.cbam3 = CBAMBlock3D(init_channels*4)
        self.cbam4 = CBAMBlock3D(init_channels*8)
        
        self.avgpool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(init_channels*8 * block.expansion, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(self.in_channels, out_channels * block.expansion, 
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(out_channels * block.expansion)
            )
            
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.cbam1(x)
        
        x = self.layer2(x)
        x = self.cbam2(x)
        
        x = self.layer3(x)
        x = self.cbam3(x)
        
        x = self.layer4(x)
        x = self.cbam4(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.sigmoid(x)
        
        return x

In [31]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25, device='cuda'):
    best_val_loss = float('inf')
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in tqdm(train_loader, desc='Training'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                preds = (outputs > 0.5).float()
                loss.backward()
                optimizer.step()
                
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Validation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in tqdm(val_loader, desc='Validation'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                preds = (outputs > 0.5).float()
                
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / len(val_loader.dataset)
        epoch_acc = running_corrects.double() / len(val_loader.dataset)
        
        history['val_loss'].append(epoch_loss)
        history['val_acc'].append(epoch_acc.item())
        
        print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Step the scheduler
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(epoch_loss)
        else:
            scheduler.step()
        
        # Save best model
        if epoch_loss < best_val_loss:
            best_val_loss = epoch_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print('Best model saved!')
            
        print()
        
    return history

In [32]:
def main():
    # Hyperparameters
    patch_size = 64
    batch_size = 16
    num_epochs = 7
    learning_rate = 0.001
    negative_sample_ratio = 0.1  # To balance the dataset
    
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # Split data into train and validation
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['class'])
    
    # Create datasets
    train_dataset = Luna16Dataset(train_data, patch_size=patch_size, sample_negative_ratio=negative_sample_ratio)
    val_dataset = Luna16Dataset(val_data, patch_size=patch_size, sample_negative_ratio=negative_sample_ratio)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    # Initialize model
    model = ResNet3D_CBAM(BasicBlock3D, [2, 2, 2, 2]).to(device)
    
    # Loss and optimizer
    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    
    # Train the model
    history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, 
                         num_epochs=num_epochs, device=device)
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Acc')
    plt.plot(history['val_acc'], label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()
    
    # Save final model
    torch.save(model.state_dict(), 'final_model.pth')
    print('Final model saved!')

if __name__ == '__main__':
    main()

Using device: cpu


/usr/local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/7
----------


Training: 100%|██████████| 3905/3905 [55:25<00:00,  1.17it/s] 


Train Loss: 0.1550 Acc: 0.9646


Validation: 100%|██████████| 977/977 [13:58<00:00,  1.17it/s]


Val Loss: 0.1525 Acc: 0.9648
Best model saved!

Epoch 2/7
----------


Training:   2%|▏         | 72/3905 [01:37<1:26:40,  1.36s/it]


KeyboardInterrupt: 

In [34]:
def evaluate_model(model, test_loader, device='cuda'):
    model.eval()
    running_corrects = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Evaluating'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            preds = (outputs > 0.5).float()
            
            running_corrects += torch.sum(preds == labels.data)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    accuracy = running_corrects.double() / len(test_loader.dataset)
    print(f'Test Accuracy: {accuracy:.4f}')
    
    return np.array(all_preds), np.array(all_labels)

# Example usage:
# test_preds, test_labels = evaluate_model(model, test_loader, device)

NameError: name 'model' is not defined